In [1]:
import sqlite3

conn = sqlite3.connect('pokemon.db')
cursor = conn.cursor()

cuarta_generacion = 'cuarta_generacion'
cuarta_generacion_copia = 'cuarta_generacion_copia'

# Obtener los nombres de columnas de la tabla original
cursor.execute(f"PRAGMA table_info({cuarta_generacion})")
columns = cursor.fetchall()

# Filtrar las columnas para no incluir la columna `id_generacion`
column_names = [col[1] for col in columns if col[1] != 'id_generacion']
column_types = [col[2] for col in columns if col[1] != 'id_generacion']
'''
/// column_names = [col[1] for col in columns if col[1] != 'id_generacion'] ///

columns: Es una lista de tuplas obtenida de la consulta PRAGMA table_info(<table_name>).
Cada tupla representa una fila de la información de una columna de la tabla
y contiene detalles como el nombre de la columna, el tipo de datos, si es nula, etc.

    Una tupla típica podría ser ('id_generacion', 'INTEGER', 1, 0, None, 1) para una columna de ID autoincremental.

    col[1]: En cada tupla en columns, col[1] representa el NOMBRE de la columna.
    if col[1] != 'id_generacion': Esto filtra las columnas para excluir la columna llamada id_generacion.

    column_names: La lista resultante contiene los nombres de todas las columnas excepto id_generacion.

En resumen, esta línea genera una lista de nombres de columnas excluyendo la columna id_generacion.

column_types = [col[2] for col in columns if col[1] != 'id_generacion']

    col[2]: En cada tupla en columns, col[2] representa el TIPO DE DATO de la columna (por ejemplo, INTEGER, TEXT, REAL).

    if col[1] != 'id_generacion': Similar al filtro anterior, esta línea también excluye la columna id_generacion pero en lugar de obtener los nombres, obtiene los tipos de datos correspondientes.
    column_types: La lista resultante contiene los tipos de datos de todas las columnas excepto id_generacion.

En resumen, esta línea genera una lista de tipos de datos de las columnas excluyendo la columna id_generacion.

/// (col[0]: El número de columna (el índice de la columna en la tabla, comenzando desde 0). 
Este valor indica la posición de la columna dentro de la tabla). //
'''


# Definir los tipos de columnas en la nueva tabla
def get_column_definition(name, col_type):
    # Considerar los tipos de datos y definir TEXT para texto con dígitos o caracteres especiales
    if col_type in ('INTEGER', 'REAL'):
        return f'{name} {col_type}'
    else:
        return f'{name} TEXT'


# Crear las definiciones de columna. Sust de par. name, col_type. Por arg. column_names, column_types 
column_definitions = [get_column_definition(name, col_type) for name, col_type in zip(column_names, column_types)]

# Eliminar la tabla copia si ya existe
drop_table_sql = f'DROP TABLE IF EXISTS "{cuarta_generacion_copia}";'
try:
    cursor.execute(drop_table_sql)
except sqlite3.OperationalError as e:
    print(f'Error al eliminar la tabla: {e}')
    conn.close()
    raise

# Crear la nueva tabla con columna autoincremental
create_table_sql = f'''
CREATE TABLE {cuarta_generacion_copia}(
    id_generacion INTEGER PRIMARY KEY AUTOINCREMENT,
    {', '.join(column_definitions)}
);
'''

print("SQL de creación de tabla:")
print(create_table_sql)  # Imprimir el SQL para depuración

try:
    cursor.execute(create_table_sql)
except sqlite3.OperationalError as e:
    print(f'Error al crear la tabla: {e}')
    conn.close()
    raise

# Copiar datos desde la tabla original
columns_str = ', '.join(column_names)
values_str = ', '.join(['?' for _ in column_names])

insert_data_sql = f'''
INSERT INTO {cuarta_generacion_copia} ({columns_str})
SELECT {columns_str}
FROM {cuarta_generacion};
'''
print("SQL de inserción de datos:")
print(insert_data_sql)  # Imprimir el SQL para depuración

try:
    cursor.execute(insert_data_sql)
except sqlite3.OperationalError as e:
    print(f'Error al insertar datos: {e}')

print('*************Tabla exportada con exito*************')
conn.commit()
conn.close()

SQL de creación de tabla:

CREATE TABLE cuarta_generacion_copia(
    id_generacion INTEGER PRIMARY KEY AUTOINCREMENT,
    id_global INTEGER, nombre TEXT, tipo_1 TEXT, tipo_2 TEXT, japones TEXT, romanizado TEXT, rom_oficial TEXT, g_huevo_1 TEXT, g_huevo_2 TEXT
);

SQL de inserción de datos:

INSERT INTO cuarta_generacion_copia (id_global, nombre, tipo_1, tipo_2, japones, romanizado, rom_oficial, g_huevo_1, g_huevo_2)
SELECT id_global, nombre, tipo_1, tipo_2, japones, romanizado, rom_oficial, g_huevo_1, g_huevo_2
FROM cuarta_generacion;

*************Tabla exportada con exito*************
